In [ ]:
!CMAKE_ARGS="-DLLAMA_METAL=on" pip install llama-cpp-python

In [ ]:
import duckdb
from wurlitzer import pipes
from duckdb_utils import generate_sql
from llama_cpp import Llama

In [ ]:
# Load Model
with pipes() as (out, err):
    client = Llama(
        model_path="DuckDB-NSQL-7B-v0.1-q8_0.gguf",
        n_ctx=2048,
        verbose=False,
    )

In [ ]:
# if salaries.db exists, remove it
!rm -f salaries.db*

# Instantiate a DuckDB connection
connection = duckdb.connect('salaries.db')
# Read the CSV file
connection.execute("CREATE TABLE salary AS SELECT * FROM read_csv('data/salaries.csv', AUTO_DETECT=TRUE)")


In [ ]:
questions = [
            "get average salary by experience level",
            "Group average salary by job_title where company_location is US?",
            "Average salary by job titles? Make sure the output is sorted in descending order.",
            "Highest paid job title where company_location is US? Show the job title and the salary.",
            "Top 10 highest paid job titles where company_locationin is US? Show the job titles and the salaries.",
            ]

In [ ]:
#sql_query = generate_sql(question[4], connection, client)
#connection.execute(sql_query).fetchdf()

sql_query = []
# get the query for each question
for question in questions:
    sql_query.append(generate_sql(question, connection, client))
# loop through the queries and execute them together with the question
for i in range(len(questions)):
    print(questions[i])
    print(connection.execute(sql_query[i]).fetchdf())
    print("\n\n")

In [ ]:
connection.close()
# remove salaries.db
!rm salaries.db*